## Load data

In [1]:
import pandas as pd

In [2]:
df_base = pd.read_csv('../data/pjm_pivot.csv', index_col=0, parse_dates=True)
df_base.head()

,AE,AEP,AP,ATSI,BC,CE,DAY,DEOK,DOM,DPL,DUQ,EKPC,JC,ME,PE,PEP,PL,PN,PS,RECO
period,,,,,,,,,,,,,,,,,,,,
2018-07-01 05:00:00,1301,12187,4384,6960,3134,12136,1619,2895,9775,1919,1466,1209,2388,1442,4397,3165,3835,1611,5009,186
2018-07-01 06:00:00,1314,11946,4391,6762,3139,11872,1605,2857,9787,1950,1455,1197,2416,1465,4423,3156,3901,1641,4990,187
2018-07-01 07:00:00,1410,12664,4757,6670,3377,11992,1707,2997,10453,2160,1528,1273,2644,1605,4743,3332,4232,1728,5267,202
2018-07-01 08:00:00,1567,14069,5308,7065,3788,12860,1916,3331,11734,2470,1688,1472,3064,1784,5230,3679,4613,1899,5735,230
2018-07-01 09:00:00,1749,15610,5862,7833,4262,14212,2145,3703,13084,2765,1875,1656,3569,1972,5752,4085,5014,2055,6299,259


## Select region

In [3]:
region = 'AE'

In [4]:
# Parameters
region = "PS"


In [5]:
df = df_base.loc[:, [region]]
df

,PS
period,
2018-07-01 05:00:00,5009
2018-07-01 06:00:00,4990
2018-07-01 07:00:00,5267
2018-07-01 08:00:00,5735
2018-07-01 09:00:00,6299
...,...
2023-11-01 00:00:00,4847
2023-11-01 01:00:00,4772
2023-11-01 02:00:00,4627


## Data Preprocessing

### Scale Data

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
data_norm = scaler.fit_transform(df)

### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_norm, test_size=0.2, shuffle=False)

### Create Sequences

In [8]:
import numpy as np

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 24 # Use 24 hours prior to predict the following hour
X_train, y_train = create_sequences(train, sequence_length)
X_test, y_test = create_sequences(test, sequence_length)

## Modelling

### Design NN Architecture

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

2023-11-26 18:35:36.737449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Train Model

In [10]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train, y_train, epochs=50, batch_size=64, verbose=0,
    validation_data=(X_test, y_test), callbacks=[early_stop])

### Calculate Predictions

In [11]:
y_pred = model.predict(X_test)
y_pred

  1/280 [..............................] - ETA: 2:37

 20/280 [=>............................] - ETA: 0s  

 40/280 [===>..........................] - ETA: 0s

 60/280 [=====>........................] - ETA: 0s

 80/280 [=======>......................] - ETA: 0s

...

247/280 [=========================>....] - ETA: 0s

258/280 [==========================>...] - ETA: 0s

278/280 [============================>.] - ETA: 0s

280/280 [==============================] - 1s 3ms/step


array([[0.27263695],
       [0.25183353],
       [0.232858  ],
       ...,
       [0.25692716],
       [0.23219375],
       [0.19682865]], dtype=float32)

### Comparison: Real Data & Prediction

#### Descale Data

In [12]:
y_pred_scaled_inverse = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)

#### Create DataFrame

In [13]:
dic_pred = {
    'y_pred': y_pred.flatten(),
    'y_pred_scaled_inverse': y_pred_scaled_inverse.flatten(),
    'y_true': y_true.flatten(),
}

df_pred = pd.DataFrame(dic_pred)
df_pred

,y_pred,y_pred_scaled_inverse,y_true
0,0.272637,4745.419922,4753.0
1,0.251834,4590.871094,4620.0
2,0.232858,4449.902344,4419.0
3,0.202208,4222.206543,4202.0
4,0.177930,4041.845215,4013.0
...,...,...,...
8952,0.292677,4894.300293,4847.0
8953,0.267902,4710.246094,4772.0
8954,0.256927,4628.711914,4627.0
8955,0.232194,4444.967773,4393.0


#### Evaluate Model

In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
mean_squared_error(df_pred.y_true, df_pred.y_pred_scaled_inverse, squared=False)

83.61483448321799

In [16]:
model.save(f'../models/{region}.keras')